In [7]:
from PIL import Image
import os
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, datasets
import os
from torch.utils.data import DataLoader, Dataset
import math
from torchvision.transforms import ColorJitter, Normalize
from torch.utils.data import ConcatDataset
from torch.utils.data import Subset
from torch.utils.data.sampler import SubsetRandomSampler
from torch.utils.data import DataLoader, random_split
import numpy as np
from tqdm import tqdm


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
low_res_folder = "./drive/MyDrive/Upscaling/mensch_low_res"
high_res_folder = "./drive/MyDrive/Upscaling/mensch"


# === creating dataset with all images ===
class CustomDataset(Dataset):
    def __init__(self, low_res_folder, high_res_folder, transform=None):
        self.low_res_folder = low_res_folder
        self.high_res_folder = high_res_folder
        self.low_res_images = sorted(os.listdir(low_res_folder))
        self.high_res_images = sorted(os.listdir(high_res_folder))
        self.transform = transform

    def __len__(self):
        return len(self.low_res_images)

    def __getitem__(self, index):
        low_res_image = Image.open(os.path.join(self.low_res_folder, self.low_res_images[index]))
        high_res_image = Image.open(os.path.join(self.high_res_folder, self.high_res_images[index]))

        if self.transform is not None:
            low_res_image = self.transform(low_res_image)
            high_res_image = self.transform(high_res_image)

        return low_res_image, high_res_image

# transform to tensor & normalize
normalize = Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])

base_transform = transforms.Compose([
    transforms.ToTensor(),
    normalize
])

# original dataset
dataset = CustomDataset(low_res_folder, high_res_folder, transform=base_transform)

### SRCNN

In [ ]:
# SRCNN model
class SRCNN(nn.Module):
    def __init__(self):
        super(SRCNN, self).__init__()
        self.interpolation = nn.Upsample(scale_factor=4, mode='bicubic')
        self.conv1 = nn.Conv2d(3, 64, kernel_size=9, stride=1, padding=4)
        self.relu1 = nn.ReLU()
        self.conv2 = nn.Conv2d(64, 32, kernel_size=1, stride=1, padding=0)
        self.relu2 = nn.ReLU()
        self.conv3 = nn.Conv2d(32, 3, kernel_size=5, stride=1, padding=2)
        self.relu3 = nn.ReLU()

    def forward(self, x):
        x = self.interpolation(x)
        x = self.relu1(self.conv1(x))
        x = self.relu2(self.conv2(x))
        x = self.relu3(self.conv3(x))
        return x

# Training hardware
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# instance of the CNN model
model = SRCNN().to(device)

# hyperparameters
learning_rate = 0.001
num_epochs = 10

# loss function and optimizer
criterion = nn.MSELoss() # note: standard MSE is used, PSNR normally not used for training (just as metric at the end)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training process
for epoch in range(num_epochs):
    for input_data, desired_data in train_loader:
        # Move input and desired images to device
        input_data = input_data.to(device)
        desired_data = desired_data.to(device)

        # Forward pass
        output_images = model(input_data)

        # Calculate loss
        loss = criterion(output_images, desired_data)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # Print training loss per epoch
    psnr = 10 * math.log10(1 / loss.item())
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}, PSNR: {psnr}")

    # Validating the model (on validation data)
    for input_data, desired_data in val_loader:
        # Move input and desired images to device
        input_data = input_data.to(device)
        desired_data = desired_data.to(device)

        # Forward pass
        output_images = model(input_data)

        # Calculate loss
        loss = criterion(output_images, desired_data)

    # Print training loss per epoch
    psnr = 10 * math.log10(1 / loss.item())

    print(f"Loss (validation): {loss.item():.4f}, PSNR (validation): {psnr}")

# saving the model
torch.save(model.state_dict(), "SRCNN.pth")

### FSRCNN

In [8]:
class FSRCNN(nn.Module):
    def __init__(self, d=56, s=12, m=4):
        super(FSRCNN, self).__init__()
        # Feature Extraction
        self.conv1 = nn.Conv2d(3, d, kernel_size=5, padding=2)
        self.relu1 = nn.PReLU(d)
        # Shrinking
        self.conv2 = nn.Conv2d(d, s, kernel_size=1)
        self.relu2 = nn.PReLU(s)
        # Non-linear Mapping
        self.mapping = nn.Sequential(*[nn.Sequential(
            nn.Conv2d(s, s, kernel_size=3, padding=1),
            nn.PReLU(s)
        ) for _ in range(m)])
        # Expanding
        self.conv3 = nn.Conv2d(s, d, kernel_size=1)
        self.relu3 = nn.PReLU(d)
        # Deconvolution
        self.deconv = nn.ConvTranspose2d(d, 3, kernel_size=9, stride=5, padding=4, output_padding=4)

    def forward(self, x):
        x = self.relu1(self.conv1(x))
        x = self.relu2(self.conv2(x))
        x = self.mapping(x)
        x = self.relu3(self.conv3(x))
        x = self.deconv(x)
        return x

In [13]:
# Assuming that dataset is your dataset
dataset_size = len(dataset)
indices = list(range(dataset_size))

# Cross-validation setup
num_folds = 5
fold_len = np.floor(len(indices)/num_folds).astype('int')

batch_size = 16
total_psnr = 0
for fold in range(num_folds):
    # Initialize train and validation samplers
    train_indices = indices[:fold*fold_len] + indices[(fold+1)*fold_len:]
    val_indices = indices[fold*fold_len:(fold+1)*fold_len]

    train_sampler = SubsetRandomSampler(train_indices)
    val_sampler = SubsetRandomSampler(val_indices)

    train_loader = DataLoader(dataset, batch_size=batch_size, sampler=train_sampler)
    val_loader = DataLoader(dataset, batch_size=batch_size, sampler=val_sampler)

    # Training hardware
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # instance of the CNN model
    model = FSRCNN().to(device)

    # hyperparameters
    learning_rate = 0.001
    num_epochs = 1

    # loss function and optimizer
    criterion = nn.MSELoss() # note: standard MSE is used, PSNR normally not used for training (just as metric at the end)
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    # Training process
    print("hi")
    for epoch in range(num_epochs):
        a=0
        for input_data, desired_data in train_loader:
            a+=1
            print(a)
            # Move input and desired images to device
            input_data = input_data.to(device)
            desired_data = desired_data.to(device)

            # Forward pass
            output_images = model(input_data)

            # Calculate loss
            loss = criterion(output_images, desired_data)

            # Backward and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        # Print training loss per epoch
        psnr = 10 * math.log10(1 / loss.item())
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}, PSNR: {psnr}")

        # Validating the model (on validation data)
        val_loss = 0
        number_batches = 0
        for input_data, desired_data in val_loader:
            number_batches += 1
            # Move input and desired images to device
            input_data = input_data.to(device)
            desired_data = desired_data.to(device)

            # Forward pass
            output_images = model(input_data)

            # Calculate loss
            loss = criterion(output_images, desired_data)
            val_loss += loss.item()

        val_loss_avg = val_loss / number_batches

        # Print training loss per epoch
        psnr = 10 * math.log10(1 / val_loss_avg)
        total_psnr += psnr

        print(f"Loss (validation): {val_loss_avg:.4f}, PSNR (validation): {psnr}")

total_psnr = total_psnr/num_folds
print(f"Total PSNR:{total_psnr}")

hi
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
Epoch [1/1], Loss: 0.0835, PSNR: 10.784431398641726
Loss (validation): 0.0705, PSNR (validation): 11.515995725847901
hi
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
Epoch [1/1], Loss: 0.0670, PSNR: 11.742103637982591
Loss (validation): 0.0682, PSNR (validation): 11.6623312524994
hi
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
Epoch [1/1], Loss: 0.0704, PSNR: 11.522969217833621
Loss (validation): 0.0762, PSNR (validation): 11.1801475655

In [11]:
range(5)

range(0, 5)

### Bicubic

In [ ]:
class bicubic(nn.Module):
    def __init__(self):
        super(bicubic, self).__init__()
        self.interpolation = nn.Upsample(scale_factor=4, mode='bicubic')

    def forward(self, x):
        x = self.interpolation(x)
        return x

# Training hardware
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# instance of the CNN model
model = bicubic().to(device)

# loss function and optimizer
criterion = nn.MSELoss() # note: standard MSE is used, PSNR normally not used for training (just as metric at the end)

# Validating the model (on validation data)
for input_data, desired_data in val_loader:
    # Move input and desired images to device
    input_data = input_data.to(device)
    desired_data = desired_data.to(device)

    # Forward pass
    output_images = model(input_data)

    # Calculate loss
    loss = criterion(output_images, desired_data)

# Print training loss per epoch
psnr = 10 * math.log10(1 / loss.item())

print(f"Loss (validation): {loss.item():.4f}, PSNR (validation): {psnr}")


# saving the model
torch.save(model.state_dict(), "bicubic.pth")


Loss (validation): 0.0114, PSNR (validation): 19.41890646298794
